# Part 1: Array-oriented programming

## What is "array-oriented programming"?

In [4]:
import numpy as np

## What array-oriented programming is good for

### The grandfather: APL

In [3]:
%%html
<div style="overflow: hidden;"><iframe src="https://tryapl.org/" width="100%" height="600" scrolling="no" style="border: none;"></div>

**Quizlet:** Translate the following NumPy operations into APL.

In [9]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [12]:
np.sum(np.arange(10))

45

In [11]:
np.cumsum(np.arange(10))

array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])

In [35]:
%%html
<!-- This will only work on the day of the live tutorial. -->
<div style="overflow: hidden;"><iframe src="https://app.sli.do/event/rbr8JR3hY4WEZ9CpWm94Xg/embed/polls/d92f941a-23fc-494d-a18b-8163205dc779" width="100%" height="280" scrolling="no" style="border: none;"></div>

### Distributions and interactivity

## NumPy

<img src="../img/Numpy_Python_Cheat_Sheet.svg" width="100%">